**Make sure to upload the Database schema here**

In [ ]:
# Install necessary libraries
!pip install transformers peft accelerate bitsandbytes torch datasets tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
import os
import json
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, PeftModel
from datasets import Dataset
import numpy as np
from transformers import Trainer

# Load the database schema from the uploaded file
with open('database_schema.json', 'r') as f:
    db_schema_dict = json.load(f)

# Convert the schema dict to a CREATE TABLE format
schema_str = ""
for table_name, columns in db_schema_dict.items():
    schema_str += f"CREATE TABLE {table_name} (\n"
    column_definitions = []
    for col_name, col_type in columns.items():
        column_definitions.append(f"  {col_name} {col_type}")
    schema_str += ",\n".join(column_definitions)
    schema_str += "\n);\n\n"

print("Loaded schema with tables:", ", ".join(db_schema_dict.keys()))

Loaded schema with tables: users, student, course, students_enrolled, courses_enrolled, assignments, mentee


In [ ]:
def create_schema_examples(schema):
    """Create fine-tuning examples from your database schema"""
    examples = []

    # Create various example queries for your specific schema
    example_queries = [
        {
            "natural_language": "Find all students in the Computer Science department",
            "sql_query": "SELECT * FROM student WHERE dept = 'Computer Science';"
        },
        {
            "natural_language": "List all courses enrolled by student with email student@example.com",
            "sql_query": "SELECT course_name, course_id FROM courses_enrolled WHERE email = 'student@example.com';"
        },
        {
            "natural_language": "Show students with assignment scores above 90",
            "sql_query": "SELECT name, email, roll_no FROM assignments WHERE assignment0 > 90 OR assignment1 > 90 OR assignment2 > 90;"
        },
        {
            "natural_language": "Find all mentees assigned to mentor 'Dr. Smith'",
            "sql_query": "SELECT name, email, roll_no FROM mentee WHERE mentor_name = 'Dr. Smith';"
        },
        {
            "natural_language": "Count how many students are enrolled in each course",
            "sql_query": "SELECT course_name, COUNT(*) as student_count FROM students_enrolled GROUP BY course_name ORDER BY student_count DESC;"
        },
        {
            "natural_language": "List users with admin role",
            "sql_query": "SELECT name, email FROM users WHERE role = 'admin';"
        },
        {
            "natural_language": "Find students who haven't submitted assignment 2",
            "sql_query": "SELECT name, email FROM assignments WHERE assignment2 IS NULL;"
        },
        {
            "natural_language": "List courses with their enrolled student count",
            "sql_query": "SELECT course.course_name, course.course_id, COUNT(students_enrolled.email) AS enrolled_count FROM course LEFT JOIN students_enrolled ON course.course_id = students_enrolled.course_id GROUP BY course.course_name, course.course_id;"
        },
        {
            "natural_language": "Show all female students in their final year",
            "sql_query": "SELECT name, email, roll_no FROM student WHERE gender = 'female' AND year = 4;"
        },
        {
            "natural_language": "Find the average score for each assignment",
            "sql_query": "SELECT AVG(assignment0) as avg_assignment0, AVG(assignment1) as avg_assignment1, AVG(assignment2) as avg_assignment2 FROM assignments;"
        },
        {
        "natural_language": "Find the top 5 students with the highest average assignment score",
        "sql_query": "SELECT name, email, roll_no, (assignment0 + assignment1 + assignment2) / 3 AS avg_score FROM assignments ORDER BY avg_score DESC LIMIT 5;"
    },
    {
        "natural_language": "List all students who have enrolled in both 'Database Systems' and 'Machine Learning'",
        "sql_query": "SELECT s.name, s.email FROM students_enrolled se1 JOIN students_enrolled se2 ON se1.email = se2.email WHERE se1.course_name = 'Database Systems' AND se2.course_name = 'Machine Learning';"
    },
    {
        "natural_language": "Find students who haven't enrolled in any courses",
        "sql_query": "SELECT s.name, s.email FROM student s LEFT JOIN students_enrolled se ON s.email = se.email WHERE se.email IS NULL;"
    },
    {
        "natural_language": "Get the number of students per department, sorted in descending order",
        "sql_query": "SELECT dept, COUNT(*) as student_count FROM student GROUP BY dept ORDER BY student_count DESC;"
    },
    {
        "natural_language": "List the mentors who have more than 5 mentees",
        "sql_query": "SELECT mentor_name, COUNT(*) as mentee_count FROM mentee GROUP BY mentor_name HAVING COUNT(*) > 5;"
    },
    {
        "natural_language": "Find students who have completed all assignments (i.e., none are NULL)",
        "sql_query": "SELECT name, email FROM assignments WHERE assignment0 IS NOT NULL AND assignment1 IS NOT NULL AND assignment2 IS NOT NULL;"
    },
    {
        "natural_language": "Retrieve the details of students who scored below 40 in at least one assignment",
        "sql_query": "SELECT name, email, roll_no FROM assignments WHERE assignment0 < 40 OR assignment1 < 40 OR assignment2 < 40;"
    },
    {
        "natural_language": "Find the most popular course with the highest number of enrollments",
        "sql_query": "SELECT course_name, COUNT(*) as enrolled_students FROM students_enrolled GROUP BY course_name ORDER BY enrolled_students DESC LIMIT 1;"
    },
    {
        "natural_language": "Calculate the pass percentage of students (consider pass if avg score is 50 or more)",
        "sql_query": "SELECT (COUNT(CASE WHEN (assignment0 + assignment1 + assignment2) / 3 >= 50 THEN 1 END) * 100.0 / COUNT(*)) AS pass_percentage FROM assignments;"
    },
    {
        "natural_language": "Find students who have enrolled in a course but haven't submitted any assignments",
        "sql_query": "SELECT s.name, s.email FROM students_enrolled s LEFT JOIN assignments a ON s.email = a.email WHERE a.email IS NULL;"
    }
    ]

    for ex in example_queries:
        prompt = f"""
### Instructions:
Convert the following natural language query into a SQL query.

### Database Schema:
{schema}

### Query:
{ex['natural_language']}

### SQL Query:
"""
        examples.append({
            "input": prompt,
            "output": ex['sql_query']
        })

    return examples

# Create fine-tuning dataset
examples = create_schema_examples(schema_str)
dataset = Dataset.from_list(examples)
print(f"Created {len(examples)} training examples")

Created 20 training examples


In [ ]:
# Define model name
model_name = "defog/sqlcoder-7b-2"  # SQL-focused LLM

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Fix the padding token issue
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    print(f"Set padding token to: {tokenizer.pad_token}")

# Load the model in 4-bit to save memory
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    load_in_4bit=True,
    torch_dtype=torch.float16
)

# Prepare for parameter-efficient fine-tuning
model = prepare_model_for_kbit_training(model)

# Configure LoRA (Low-Rank Adaptation)
lora_config = LoraConfig(
    r=16,               # Rank of LoRA matrices
    lora_alpha=32,      # Parameter scaling
    lora_dropout=0.05,  # Dropout probability for LoRA layers
    bias="none",        # Don't train bias parameters
    task_type="CAUSAL_LM",  # Task type
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"]  # Which modules to apply LoRA to
)

# Apply LoRA to model
model = get_peft_model(model, lora_config)
print("Model loaded and prepared for LoRA fine-tuning")

tokenizer_config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/515 [00:00<?, ?B/s]

Set padding token to: </s>


...

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Model loaded and prepared for LoRA fine-tuning


In [ ]:
def tokenize_function(examples):
    # Tokenize inputs
    model_inputs = tokenizer(examples["input"], padding="max_length", truncation=True, max_length=512)

    # Tokenize outputs (labels)
    labels = tokenizer(examples["output"], padding="max_length", truncation=True, max_length=512)
    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

tokenized_dataset = dataset.map(tokenize_function, batched=True)
print("Dataset tokenized and prepared for training")

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Dataset tokenized and prepared for training


In [ ]:
  # Configure training arguments
  training_args = TrainingArguments(
      output_dir="./sql_db_model",
      per_device_train_batch_size=2,  # Increased batch size (if memory allows)
      gradient_accumulation_steps=2,  # Reduce if batch size is higher
      num_train_epochs=5,
      fp16=True,
      logging_steps=5,
      save_strategy="epoch",
      learning_rate=1e-4,  # Lower LR for better stability
      lr_scheduler_type="cosine",  # Smoother learning rate decay
      weight_decay=0.01,
      warmup_ratio=0.03,
      optim="paged_adamw_8bit",
      report_to="wandb",  # Enable logging
  )

  # Create Trainer
  trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=tokenized_dataset,
  )

  # Train the model
  print("Starting training...")
  trainer.train()
  print("Training completed!")

Starting training...


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: hemanthrajc (hemanthrajc-chennai-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we wi

Step,Training Loss
5,9.869100
10,6.719600
15,4.555000
20,3.466800
25,2.952600


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/

Training completed!


In [ ]:
# Save the LoRA adapters (these are tiny compared to full model)
model.save_pretrained("./lora_adapters")
tokenizer.save_pretrained("./lora_adapters")
print("LoRA adapters saved successfully!")

LoRA adapters saved successfully!


In [ ]:
def test_model_with_query(natural_language_query):
    """Test the model with a natural language query"""
    # Prepare the prompt
    prompt = f"""
### Instructions:
Convert the following natural language query into a SQL query.

### Database Schema:
{schema_str}

### Query:
{natural_language_query}

### SQL Query:
"""

    # Tokenize the prompt
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Generate SQL query
    with torch.no_grad():
        outputs = model.generate(
        inputs.input_ids,
        max_length=1024,  # Allow longer queries
        num_beams=5,  # Beam search for better SQL coherence
        do_sample=True,  # Enable sampling
        temperature=0.7,  # Reduce randomness for structured queries
        top_p=0.9,  # Use nucleus sampling
        top_k=50,  # Avoid unlikely tokens
        repetition_penalty=1.2,  # Prevent repetitive tokens
        pad_token_id=tokenizer.eos_token_id
    )

    # Decode the output
    generated_sql = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Return the part after "### SQL Query:"
    if "### SQL Query:" in prompt:
        generated_sql = generated_sql.split("### SQL Query:")[-1].strip()

    return generated_sql.strip()

# Test with a sample query
test_query = "Show me student named hemanth assignment details"
generated_sql = test_model_with_query(test_query)

print(f"Test Query: {test_query}")
print(f"Generated SQL: {generated_sql}")

Test Query: Show me student named hemanth assignment details
Generated SQL: SELECT a.name, a.email, a.roll_no, a.assignment0, a.assignment1, a.assignment2, a.assignment3, a.assignment4, a.assignment5, a.assignment6, a.assignment7, a.assignment8, a.assignment9, a.assignment10, a.assignment11, a.assignment12, a.created_at FROM assignments a WHERE a.name ilike '%hemanth%';


In [ ]:
# Create a download script for the model
import shutil
from google.colab import files

# Zip the LoRA adapters
print("Preparing model for download...")
!zip -r sql_db_model_lora.zip lora_adapters

# Download the zip file
files.download('sql_db_model_lora.zip')
print("Downloaded LoRA adapters (you'll need the base model to use these)")

Preparing model for download...
  adding: lora_adapters/ (stored 0%)
  adding: lora_adapters/tokenizer.model (deflated 55%)
  adding: lora_adapters/adapter_model.safetensors (deflated 9%)
  adding: lora_adapters/README.md (deflated 66%)
  adding: lora_adapters/adapter_config.json (deflated 55%)
  adding: lora_adapters/tokenizer_config.json (deflated 78%)
  adding: lora_adapters/special_tokens_map.json (deflated 71%)
  adding: lora_adapters/tokenizer.json (deflated 85%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Downloaded LoRA adapters (you'll need the base model to use these)
